In [1]:
library(brms) #predictability: Double hierearchical model
library(MCMCglmm) #HPDinterval
library(dplyr) #%>%
library(lmerTest) #lmer
#library(MuMIn) #r2
#library(merTools) #sim
library(sjPlot) #plot coeff
library(ggplot2) #ylim in plot
library(rstan) #model diagnostic

Warning message:
"le package 'brms' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Rcpp

Warning message:
"le package 'Rcpp' a été compilé avec la version R 4.1.2"
Loading 'brms' package (version 2.16.3). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attachement du package : 'brms'


L'objet suivant est masqué depuis 'package:stats':

    ar


Warning message:
"le package 'MCMCglmm' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Matrix

Warning message:
"le package 'Matrix' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : coda

Warning message:
"le package 'coda' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : ape

Warning message:
"le package 'ape' a été compilé avec la version R 4.1.2"

Attachement du package : 'MCMCglmm'


L'objet suivant est masqué depuis 'pac

In [2]:
library(ggplot2)

In [3]:
library(parallel)
my.cores = detectCores()-1 #minus 1 to avoid bocking the computer
my.cores

[1] 11

In [4]:
path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_'
#df = read.csv(file.path(path_,'df_daily_aggregatedHA.csv'), header = TRUE, sep = ",")
df = read.csv(file.path(path_,'OFH_df_MVT_simpler2.csv'), header = TRUE, sep = ",")

df$HenID = as.factor(df$HenID)   
df$PenID = as.factor(df$PenID)
df$TrackingSystemID = as.factor(df$TrackingSystemID)
df$level = as.factor(df$level) 
df$CLASS = as.factor(df$CLASS) 
df$Treatment = as.factor(df$Treatment)

#scale: mean-centering of the environmental variable so that intercepts reflect average values for the HenID and ind. 
#df$temperature_C_avg_scale = scale(df$temperature_C_avg, center=TRUE, scale=TRUE)

#setting reference group
contrasts(df$Treatment)
df <- df %>% mutate(Treatment = relevel(Treatment, ref = "TRAN"))
contrasts(df$Treatment)

contrasts(df$CLASS)
df <- df %>% mutate(CLASS = relevel(CLASS, ref = "LEXP"))
contrasts(df$CLASS)
print(dim(df))
summary(df)

,TRAN
OFH,0
TRAN,1


,OFH
TRAN,0
OFH,1


,LEXP,LEXPLOST,MEXP,MEXPLOST,NewAfterEpi
EPI,0,0,0,0,0
LEXP,1,0,0,0,0
LEXPLOST,0,1,0,0,0
MEXP,0,0,1,0,0
MEXPLOST,0,0,0,1,0
NewAfterEpi,0,0,0,0,1


,EPI,LEXPLOST,MEXP,MEXPLOST,NewAfterEpi
LEXP,0,0,0,0,0
EPI,1,0,0,0,0
LEXPLOST,0,1,0,0,0
MEXP,0,0,1,0,0
MEXPLOST,0,0,0,1,0
NewAfterEpi,0,0,0,0,1


[1] 1968   88


     HenID             level      verification_daily_total_duration
 hen_102:  31   2021-01-10:  69   Min.   :54000                    
 hen_112:  31   2021-01-17:  69   1st Qu.:54000                    
 hen_134:  31   2021-01-24:  69   Median :54000                    
 hen_153:  31   2020-11-29:  68   Mean   :54000                    
 hen_166:  31   2020-12-06:  68   3rd Qu.:54000                    
 hen_169:  31   2020-12-13:  68   Max.   :54000                    
 (Other):1782   (Other)   :1557                                    
 verification_daily_total_nbr_hour  dur_values        perc_duration_3_Zone
 Min.   :15                        Length:1968        Min.   : 0.00       
 1st Qu.:15                        Class :character   1st Qu.:12.00       
 Median :15                        Mode  :character   Median :21.00       
 Mean   :15                                           Mean   :21.06       
 3rd Qu.:15                                           3rd Qu.:30.00       
 Max. 

In [5]:
#df$Matrix_proportions = data.matrix(df[,c('propindoor_duration_2_Zone','propindoor_duration_3_Zone', 'propindoor_duration_4_Zone', 'propindoor_duration_5_Zone')])

In [6]:
iter_ = 10000
iter_wu = 6000
bind <- function(...) cbind(...) #cbind is reserve for multivariate in brms!
#put 4 as reference!
frml = bf(bind(propindoor_duration_4_Zone, propindoor_duration_2_Zone, propindoor_duration_3_Zone,
               propindoor_duration_5_Zone) ~ WIB + Treatment + WIB:Treatment + (1|HenID))
fit = brm(frml, data = df, family = dirichlet(link = 'logit', link_phi = 'log'), 
         iter = iter_, warmup = iter_wu, thin = 10, chains = 3, cores = my.cores, seed = 12345)
summary(fit)
save(fit, file = file.path(path_, 'OFH_dirichlet_TESTmorebirds.rda'))
#load(file.path(path_,"OFH_dirichlet_TEST.rda")) 

Compiling Stan program...

Start sampling



 Family: dirichlet 
  Links: mupropindoorduration2Zone = logit; mupropindoorduration3Zone = logit; mupropindoorduration5Zone = logit; phi = identity 
Formula: bind(propindoor_duration_4_Zone, propindoor_duration_2_Zone, propindoor_duration_3_Zone, propindoor_duration_5_Zone) ~ WIB + Treatment + WIB:Treatment + (1 | HenID) 
   Data: df (Number of observations: 1968) 
  Draws: 3 chains, each with iter = 1000; warmup = 600; thin = 10;
         total post-warmup draws = 120

Group-Level Effects: 
~HenID (Number of levels: 80) 
                                        Estimate Est.Error l-95% CI u-95% CI
sd(mupropindoorduration2Zone_Intercept)     0.74      0.06     0.63     0.88
sd(mupropindoorduration3Zone_Intercept)     0.82      0.07     0.69     0.97
sd(mupropindoorduration5Zone_Intercept)     0.99      0.09     0.84     1.17
                                        Rhat Bulk_ESS Tail_ESS
sd(mupropindoorduration2Zone_Intercept) 1.00     1258     1155
sd(mupropindoorduration3Zone_Intercep

In [7]:
#create data to predict on
set.seed(1234)
df_newdata = expand.grid(WIB=c(1,5,10,20,30, 40), Treatment=c("OFH",'TRAN'), HenID=unique(df$HenID)) 
dim(df_newdata)
head(df_newdata, 2)
df_pred = predict(fit, newdata=df_newdata, interval='confidence') #marginal=fit$Random$formula, 
dim(df_pred)
write.csv(df_pred, file.path(path_,"df_pred_Test2.csv"), row.names = FALSE)
write.csv(df_newdata, file.path(path_,"df_pred_Test_newdata2.csv"), row.names = FALSE)

head(df_pred,2)
df_newdata$predicted = df_pred
dim(df_newdata)
head(df_newdata,3)

[1] 960   3

,WIB,Treatment,HenID
,<dbl>,<fct>,<fct>
1,1,OFH,hen_131
2,5,OFH,hen_131


[1] 960   4   4

, , propindoor_duration_4_Zone

       Estimate  Est.Error        Q2.5     Q97.5
[1,] 0.07953945 0.06590077 0.003944661 0.2489706
[2,] 0.07932416 0.06339266 0.004270813 0.2383762

, , propindoor_duration_2_Zone

      Estimate Est.Error      Q2.5     Q97.5
[1,] 0.3930138 0.1269115 0.1492922 0.6453489
[2,] 0.3973061 0.1235577 0.1715759 0.6540014

, , propindoor_duration_3_Zone

      Estimate Est.Error      Q2.5     Q97.5
[1,] 0.3681430 0.1281427 0.1457389 0.6376483
[2,] 0.3680101 0.1282005 0.1332507 0.6256110

, , propindoor_duration_5_Zone

      Estimate  Est.Error       Q2.5     Q97.5
[1,] 0.1593037 0.09632781 0.02216005 0.3917352
[2,] 0.1553596 0.09535355 0.01946204 0.3686306

[1] 960   4

,WIB,Treatment,HenID,predicted
,<dbl>,<fct>,<fct>,<dbl>
1,1,OFH,hen_131,0.07953945
2,5,OFH,hen_131,0.07932416
3,10,OFH,hen_131,0.07948142
